In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

In [ ]:
df.head()

In [ ]:
# Rename the columns name which have to required change.
df=df.rename(columns={'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1':'nb_1', 
                      'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2':'nb_2'})
df.columns

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#Count of target variable
sns.countplot(df['Attrition_Flag'])
plt.title('Count of Attrition_Flag')

From above graph we clearly note that target varible is imblanced.

In [ ]:
#Lets encode the catagorical variable
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Attrition_Flag'] = labelencoder.fit_transform(df['Attrition_Flag'])
df['Gender'] = labelencoder.fit_transform(df['Gender'])
df['Education_Level'] = labelencoder.fit_transform(df['Education_Level'])
df['Marital_Status'] = labelencoder.fit_transform(df['Marital_Status'])
df['Card_Category'] = labelencoder.fit_transform(df['Card_Category'])
df['Income_Category'] = labelencoder.fit_transform(df['Income_Category'])

In [ ]:
df.drop('CLIENTNUM', axis = 1, inplace = True)#As it is no need for use

In [ ]:
df.head()

In [ ]:
#Correlation and Heat map
corr = df.corr()
colormap = sns.diverging_palette(220, 10, as_cmap = True)
plt.figure(figsize = (12,12))
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot=True,fmt='.2f',linewidths=0.30,
            cmap = colormap, linecolor='white')
plt.title('Correlation of df Features', y = 1.05, size=15)

In [ ]:
#Lets look the correlation score
print (corr['Attrition_Flag'].sort_values(ascending=False), '\n')

In [ ]:
#At first we take our matrices of features
x = df.iloc[:,1:22].values
y = df.iloc[:,0:1].values

In [ ]:
#Spliting the dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
#Fitting Logistic Regression to the training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0, class_weight = 'balanced')
classifier.fit(x_train, y_train)

In [ ]:
#Predicting the test set result
y_pred = classifier.predict(x_test)

In [ ]:
#Making the Confusion Matrix, Accuracy score, Classification report
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
cr = classification_report(y_test, y_pred)
print(cm)
print(accuracy)
print(cr)

In [ ]:
#Let see the ROC curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
print('AUC: %.3f' % auc)

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.plot(fpr, tpr, marker='.')
plt.show()

From above graph we note that accuracy is 90 percent

Now we apply the neural network

In [ ]:
#Import Keras model for NN
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

In [ ]:
# define model
classifier = Sequential()
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu', input_dim = 21))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#Fitting ANN to the Traning set
classifier.fit(x_train, y_train, batch_size = 10, epochs = 100)

In [ ]:
#Evaluted the Classifier
score = classifier.evaluate(x_test, y_test,verbose=1)
print(score)

In [ ]:
#Predicting the Test set result
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)

In [ ]:
#Making the Confusion Matrix, Accuracy score, Classification report
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
cr = classification_report(y_test, y_pred)
print(cm)
print(accuracy)
print(cr)

In [ ]:
#Let see the ROC curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, y_pred)
print('AUC: %.3f' % auc)

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.plot(fpr, tpr, marker='.')
plt.show()